In [1]:

import time
from pathlib import Path

import h5py
import numpy as np
from tqdm import tqdm


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data.dataset import Dataset

import socket
import select
from loguru import logger

from model import Block_encoder_bottleneck, Block_decoder, dict_args, init_weights, FCT_Body

In [2]:
# device = torch.device("cuda:0")
device = torch.device("cpu")

In [3]:
# =======================================================================
#                                BODY
# =======================================================================

model_body = FCT_Body()
model_body.apply(init_weights)

# optimizer_body = torch.optim.AdamW(model_body.parameters(), lr=dict_args['lr'],weight_decay=dict_args['decay'])

# scheduler_body = torch.optim.lr_scheduler.ReduceLROnPlateau(
#             optimizer_body,
#             mode='min',
#             factor=dict_args['lr_factor'],
#             verbose=True,
#             threshold=1e-6,
#             patience=10,
#             min_lr=dict_args['min_lr'])

model_body.to(device)

FCT_Body(
  (block_5): Block_encoder_bottleneck(
    (layernorm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (trans): Transformer(
      (attention_output): Attention(
        (conv_q): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=same, groups=128)
        (layernorm_q): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (conv_k): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=128)
        (layernorm_k): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (conv_v): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=128)
        (layernorm_v): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=Tru

In [4]:
def send_socket_info(port=6161, message_data=None):
    import socket
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.connect(('127.0.0.1', port))
        if message_data is  None:
            raise ValueError(f"Message that is being sent to port {port} is empty")
        s.send(message_data.encode('UTF-8'))

In [5]:
recv_head = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
recv_head.bind(('127.0.0.1', 5151))


recv_tail = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
recv_tail.bind(('127.0.0.1', 8181))


In [6]:
# Forward propagation in body model

model_body.train()


try:
    bd_layer_data = None
    skip_1 = None
    skip_2 = None
    skip_3 = None
    skip_4 = None

    while True:
        # listen for incoming connections
        recv_head.listen()
        recv_tail.listen()

        # wait until either socket is ready to read
        read_sockets, _, _ = select.select([recv_head, recv_tail], [], [])
        try:
            for sock in read_sockets:
                if sock == recv_head:
                    conn, addr = recv_head.accept()
                    logger.info(f"connected body to get data from head 5151: {addr}")

                    head_fwd_file = conn.recv(1024)

                    if len(head_fwd_file) == 0:
                        logger.error("Frame not available")
                        break

                    with h5py.File(head_fwd_file, 'r') as head_fwd:
                        skip_1 = torch.tensor(head_fwd['skip1'][:], requires_grad=True).to(device)
                        skip_2 = torch.tensor(head_fwd['skip2'][:], requires_grad=True).to(device)
                        skip_3 = torch.tensor(head_fwd['skip3'][:], requires_grad=True).to(device)
                        skip_4 = torch.tensor(head_fwd['skip4'][:], requires_grad=True).to(device)

                    bd_layer_data = model_body(skip_1, skip_2, skip_3, skip_4)

                    logger.info(f"disconnected body to get data from head 5151: {addr}")

                    with h5py.File('params_and_grads/body_forward_pass.hdf5', 'w') as body_fwd:
                        for layer, data in bd_layer_data.items():
                            body_fwd.create_dataset(layer,  data=data.cpu().detach().numpy())

                    send_socket_info(7171, "params_and_grads/body_forward_pass.hdf5")

                    logger.info(f"sent the data from head to tail 7171 bd_layer_data")

                if sock == recv_tail:
                    conn, addr = recv_tail.accept()
                    logger.info(f"connected body to get data from tail 8181: {addr}")

                    tail_grad_file = conn.recv(1024)

                    if len(tail_grad_file) == 0:
                        logger.error("Frame not available")
                        break
                    
                    with h5py.File(tail_grad_file, 'r') as grad_tail:
                        skip_9_grad = torch.tensor(grad_tail['skip_9_grad'][:])
                        bd_layer_data['skip9'].backward(skip_9_grad)

                    # skip_1_grad = skip_1.grad
                    # skip_2_grad = skip_2.grad
                    # skip_3_grad = skip_3.grad
                    skip_4_grad = skip_4.grad

                    logger.info(f"disconnected body to get data from tail 8181: {addr}")

                    with h5py.File('params_and_grads/body_back_prop.hdf5', 'w') as body_bkp:
                        # body_bkp.create_dataset("skip_1_grad",  data=skip_1_grad.cpu().detach().numpy())
                        # body_bkp.create_dataset("skip_2_grad",  data=skip_2_grad.cpu().detach().numpy())
                        # body_bkp.create_dataset("skip_3_grad",  data=skip_3_grad.cpu().detach().numpy())
                        body_bkp.create_dataset("skip_4_grad",  data=skip_4_grad.cpu().detach().numpy())

                    send_socket_info(9191, 'params_and_grads/body_back_prop.hdf5')

                    logger.info(f"sent the backprop data from body to head 9999 skip 1/2/3/4 grad")

        except ConnectionResetError:
            logger.error(f"Connection error issue")
except Exception as e:
    import traceback
    traceback.print_exc()
finally:
    pass

2023-04-23 23:52:05.623 | INFO     | __main__:<module>:24 - connected body to get data from head 5151: ('127.0.0.1', 53922)


Block 5 out -> [4, 128, 7, 7]
Block 6 out -> [4, 64, 14, 14]
Block 7 out -> [4, 32, 28, 28]
Block 8 out -> [4, 16, 56, 56]


2023-04-23 23:52:40.577 | INFO     | __main__:<module>:40 - disconnected body to get data from head 5151: ('127.0.0.1', 53922)
2023-04-23 23:52:40.688 | INFO     | __main__:<module>:48 - sent the data from head to tail 7171 bd_layer_data


Block 9 out -> [4, 8, 112, 112]


2023-04-23 23:52:41.363 | INFO     | __main__:<module>:52 - connected body to get data from tail 8181: ('127.0.0.1', 54000)
2023-04-23 23:52:53.821 | INFO     | __main__:<module>:69 - disconnected body to get data from tail 8181: ('127.0.0.1', 54000)
2023-04-23 23:52:53.876 | INFO     | __main__:<module>:79 - sent the backprop data from body to head 9999 skip 1/2/3/4 grad
2023-04-23 23:53:17.264 | INFO     | __main__:<module>:24 - connected body to get data from head 5151: ('127.0.0.1', 54033)


...

Block 5 out -> [4, 128, 7, 7]
Block 6 out -> [4, 64, 14, 14]
Block 7 out -> [4, 32, 28, 28]
Block 8 out -> [4, 16, 56, 56]


2023-04-24 00:59:38.918 | INFO     | __main__:<module>:40 - disconnected body to get data from head 5151: ('127.0.0.1', 57676)
2023-04-24 00:59:38.934 | INFO     | __main__:<module>:48 - sent the data from head to tail 7171 bd_layer_data


Block 9 out -> [4, 8, 112, 112]


2023-04-24 00:59:39.191 | INFO     | __main__:<module>:52 - connected body to get data from tail 8181: ('127.0.0.1', 57687)
2023-04-24 01:00:01.942 | INFO     | __main__:<module>:69 - disconnected body to get data from tail 8181: ('127.0.0.1', 57687)
2023-04-24 01:00:02.105 | INFO     | __main__:<module>:79 - sent the backprop data from body to head 9999 skip 1/2/3/4 grad
2023-04-24 01:00:24.384 | INFO     | __main__:<module>:24 - connected body to get data from head 5151: ('127.0.0.1', 57757)


Block 5 out -> [4, 128, 7, 7]
Block 6 out -> [4, 64, 14, 14]
Block 7 out -> [4, 32, 28, 28]
Block 8 out -> [4, 16, 56, 56]
